## Smarthop Challenge

Extract insights and data from unstructured data like images or PDFs and GPS data

### Input

* GPS data points (JSON encoded) of a trucks trip

* BOL file (document that states what and where to pickup and where to deliver the cargo)

### Goal

Detect if the Truck passed the Pickup and Delivery Points and for how long it was there at each stop (if any)

### Expected Output

* A notebook with the answers about the Truck trip

* The output in text and structured json to model entities

## 1-Import Libraries and Set Google Variables

In [1]:
import os
import json
import base64
from datetime import datetime
from pdf2image import convert_from_path
from geopy import distance

from openai import OpenAI
import googlemaps
import google.generativeai as genai

c:\Users\sebastiancg\My GitHub (Local)\smarthop-challenge\.smarthop-challenge\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set up Google API Keys and Cloud Storage bucket
gmaps_api_key = os.getenv('GOOGLE_CLOUD_API_KEY')

genai.configure(api_key=os.getenv("GOOGLE_AI_STUDIO_API_KEY"))

my_bucket_path = 'gs://data-extraction-vision/'

## 2-Load GPS data

Use `json` dependency to load JSON file:

* Truck locations is in the `'locations'` key

In [3]:
def load_json(file_path:str, mode:str='r'):
    
    with open(file_path, mode) as json_file:
        return json.load(json_file)

In [4]:
gps_data = load_json(file_path='../GPS_BOL_Dataset/hopbackend.booked_trip_tracking_locations 2.json',
                     mode='rb')

_id = gps_data[0]['_id']['$oid']
locations = gps_data[0]['locations']

In [5]:
print(_id)

66dbc3ad6822234696fbf558


In [6]:
# Print location and date for 1st element in 'locations'
print('Location: (lat, long):')
print(locations[0]['lat'], ',', locations[0]['long'])

print('Date:')
print(locations[0]['last_date']['$date'])

Location: (lat, long):
29.96181 , -95.28558
Date:
2024-09-07T03:09:54.153Z


### 2-1 Get Location and Date from GPS data

Define methods for getting GPS location and timestamp from the JSON file:
  
* `get_gps_location(loc_dict:dict)`: Get the latitude and longitude from a location dictionary
  
* `get_gps_date(loc_dict:dict)`: Get the date in datetime format from a location dictionary

In [7]:
# Check 'locations' type
type(locations[0])

dict

In [8]:
def get_gps_location(loc_dict:dict):
    """Returns the latitude and longitude in a tuple from a location dictionary

    Args:
        loc_dict (dict): Location dictionary with latitude and longitude keys

    Returns:
        tuple: A tuple of (latitude, longitude)
        None: If the location dictionary is empty
    """

    if not loc_dict:
        return None
    return (loc_dict['lat'], loc_dict['long'])

In [9]:
# Print location for the 1st element
print(get_gps_location(locations[0]))

(29.96181, -95.28558)


In [10]:
# from datetime import datetime

def get_gps_date(loc_dict:dict):
    """Returns the date in datetime format from a location dictionary

    Args:
        loc_dict (dict): Location dictionary with date keys

    Returns:
        datetime.datetime: A datetime object representing the date
        None: If the location dictionary is empty
    """

    if not loc_dict:
        return None
    
    date_fmt = '%Y-%m-%dT%H:%M:%S.%f%z'

    str_date = loc_dict['last_date']['$date']
    
    return datetime.strptime(str_date, date_fmt)

In [11]:
# Print date for the 1st element
print(get_gps_date(locations[0]))
print(type(get_gps_date(locations[0])))

2024-09-07 03:09:54.153000+00:00
<class 'datetime.datetime'>


### 2-2 Get Distance between Locations 

#### Option A: Using Google Maps's Distance Matrix API

* Use Google Maps Platform - Distance Matrix API service to request travel distance between origins and destinations: https://developers.google.com/maps/documentation/distance-matrix/overview

  - `get_travel_distance_kms(origin:tuple, destination:tuple, mode:str="driving")`: Returns the distance between the origin and destination in kilometers

In [12]:
# import googlemaps # gmaps is defined at the beginning

gmaps = googlemaps.Client(key=gmaps_api_key)

def get_travel_distance_kms(origin:tuple, 
                            destination:tuple, 
                            mode:str="driving"):
    """
    Returns the distance between the origin and destination in kilometers.

    Args:
        origin (tuple): (latitude, longitude) of the origin
        destination (tuple): (latitude, longitude) of the destination
        mode (str): mode of transportation, default is "driving"

    Returns:
        float: distance in kilometers
    """
    distance = gmaps.distance_matrix(origins=origin,
                        destinations=destination,
                        mode=mode)["rows"][0]["elements"][0]["distance"]["value"]
    
    return distance/1000

In [13]:
# Test GPS location, date and travel distance

# dist_maps = []

# init_location = get_gps_location(locations[0])
# init_date = get_gps_date(locations[0])

# prev_loc = init_location
# prev_time = init_date
# for loc in locations[:10]:
#     loc_i = get_gps_location(loc)
#     date_i = get_gps_date(loc)
#     elapsed_time = date_i - prev_time

#     dist_travelled = get_travel_distance_kms(origin=prev_loc, 
#                                               destination=loc_i)

#     dist_maps.append(dist_travelled)

#     prev_loc = loc_i
#     prev_time = date_i

#     print('---')
#     print('Current location:\n', loc_i)
#     print('Current date:\n', date_i)
#     print('Distance from previous location:\n', dist_travelled)
#     print('Elapsed time since previous location:\n', elapsed_time)
#     print('---\n')

In [14]:
# print(dist_maps)

#### Option B: Using the `geopy` Dependency

Alternatively, Use `geopy` dependency which uses `geodesic` distance norm: https://geopy.readthedocs.io/en/stable/index.html?highlight=geodesic#module-geopy.distance
  
  - `get_geo_distance_kms(point_a:tuple, point_b:tuple)`: Returns the geodesic distance between two points in kilometers

**Decision**:

I preferred to use the `geopy` approach since we don't necessarily need travel (real road) distances to check whether a truck passed near a pickup/delivery point.

Also, computing travel distances using Google's Distance Matrix API generates cloud computing costs

In [15]:
def get_geo_distance_kms(point_a:tuple,
                         point_b:tuple):
    """Returns the geodesic distance between two points in kilometers

    Args:
        point_a (tuple): Coordinates of the first point in (lat, long) format
        point_b (tuple): Coordinates of the second point in (lat, long) format

    Returns:
        float: Geodesic distance in kilometers
    """
    
    return round(distance.distance(point_a, point_b).km, 5)

In [16]:
# Test GPS location, date and travel distance

dist_geo = []

init_location = get_gps_location(locations[0])
init_date = get_gps_date(locations[0])

prev_loc = init_location
prev_time = init_date
for loc in locations[:10]:
    loc_i = get_gps_location(loc)
    date_i = get_gps_date(loc)
    elapsed_time = date_i - prev_time

    dist_points = get_geo_distance_kms(point_a=prev_loc, 
                                       point_b=loc_i)

    dist_geo.append(dist_points)

    prev_loc = loc_i
    prev_time = date_i

    print('---')
    print('Current location:\n', loc_i)
    print('Current date:\n', date_i)
    print('Distance from previous location:\n', dist_points)
    print('Elapsed time since previous location:\n', elapsed_time)
    print('---\n')

---
Current location:
 (29.96181, -95.28558)
Current date:
 2024-09-07 03:09:54.153000+00:00
Distance from previous location:
 0.0
Elapsed time since previous location:
 0:00:00
---

---
Current location:
 (29.965555, -95.28402)
Current date:
 2024-09-07 03:10:27.601000+00:00
Distance from previous location:
 0.4416
Elapsed time since previous location:
 0:00:33.448000
---

---
Current location:
 (29.966496, -95.28652)
Current date:
 2024-09-07 03:11:05.868000+00:00
Distance from previous location:
 0.26288
Elapsed time since previous location:
 0:00:38.267000
---

---
Current location:
 (29.968079, -95.28605)
Current date:
 2024-09-07 03:11:19.562000+00:00
Distance from previous location:
 0.18125
Elapsed time since previous location:
 0:00:13.694000
---

---
Current location:
 (29.96781, -95.287056)
Current date:
 2024-09-07 03:11:53.958000+00:00
Distance from previous location:
 0.10157
Elapsed time since previous location:
 0:00:34.396000
---

---
Current location:
 (29.96785, -95.

In [17]:
print(dist_geo)

[0.0, 0.4416, 0.26288, 0.18125, 0.10157, 0.02891, 0.00022, 0.00106, 0.00067, 0.00166]


## 3-Extract Pickup and Delivery Addresses from BOL

### 3-1 Convert BOL file to image

Use `convert_from_path` from the `pdf2image` depedency.

* Save converted image in current directory with the name `'BOL-image.jpg'`

* Upload the image to Google Cloud Storage using:
  
  ```
  gcloud storage cp --recursive .\BOL-image.jpg gs://data-extraction-vision/
  ```

In [18]:
images = convert_from_path(pdf_path='../GPS_BOL_Dataset/3e13d38e-a9b6-4bd6-b42d-7af3df2977af.pdf',
                           dpi=300)

bol_img = 'BOL-image.jpg'

images[0].save(bol_img, 'JPEG')

### 3-2 Extract Addresses from Image/PDF file

First, define a method to extract addresses from json files:

* `convert_json_to_str_addr(json_addresses:dict)`

In [20]:
def convert_json_to_str_addr(json_addresses:dict):
    
    addresses = []
    for prop in json_addresses:
        
        address = []
        for addr_elem in json_addresses[prop]:
            address.append(json_addresses[prop][addr_elem])
        
        addresses.append(" ".join(map(str, address)))
    
    pickup_address, delivery_address = addresses
    
    return pickup_address, delivery_address

#### Option A: Use OpenAI API

* Use the OpenAI API to extract Pickup/Delivery information in a JSON format:
  
  - `get_pickup_delivery_addr_openai(img_path:str, schema_file:dict)`: Returns the pickup and delivery addresses in the BOL image using OpenAI's API.

In [21]:
def get_pickup_delivery_addr_openai(img_path:str,
                                    json_filename:str='json_response_openai'):
    """Returns the pickup and delivery addresses in the BOL image using OpenAI's API.

    Args:
        img_path (str): Path to the image file
        json_filename (str, optional): Name of the JSON file to save the response. Defaults to 'json_response_gemini'.

    Returns:
        str, str: Pickup and delivery addresses as strings.
    """
    
    client = OpenAI()

    with open(img_path, 'rb') as img_file:
        img_base64 = base64.b64encode(img_file.read()).decode('utf-8')

    response = client.chat.completions.create(
        model='gpt-4o',
        response_format={'type': 'json_object'},
        messages=[
            {
                'role': 'user',
                'content': [
                    {'type': 'text', 
                     'text': 'Extract the pickup and delivery addresses from this BOL image: return as JSON file with "pickup" and "delivery" with properties: street, city, state and zip code'},
                    {'type': 'image_url', 
                     'image_url': {'url': f'data:image/jpeg;base64,{img_base64}',
                                   'detail': 'high'}}
                ]
            }
        ],
        max_tokens=500
    )
    
    json_response = json.loads(response.choices[0].message.content)

    with open(f'{json_filename}.json', 'w') as outfile:
        json.dump(json_response, outfile, indent=2)

    pickup_addr, delivery_addr = convert_json_to_str_addr(json_addresses=json_response)

    return pickup_addr, delivery_addr

In [22]:
pickup, delivery = get_pickup_delivery_addr_openai(img_path=bol_img)

print('Pickup address:', pickup)
print('Delivery address:', delivery)

Pickup address: 6115 FM 1405 BAYTOWN TX 77523
Delivery address: 10300 CCOURSEY BLVD BATON ROUGE LA 70816


#### Option B: Use Google's Gemini API

You can also use Google's Gemini API: https://ai.google.dev/gemini-api/docs

* `get_pickup_delivery_addr_gemini(img_path:str)`: Returns pickup and delivery addresses from an image of a BOL. This function uses the Google Cloud AI Platform's Generative AI service to extract the pickup and delivery addresses from an image of a BOL.

In [24]:
def get_pickup_delivery_addr_gemini(img_path:str,
                                    json_filename:str='json_response_gemini'):
    """Returns pickup and delivery addresses from an image of a BOL. This function 
    uses the Google Cloud AI Platform's Generative AI service to extract the pickup 
    and delivery addresses from an image of a BOL.    

    Args:
        img_path (str): Path to the image file
        json_filename (str, optional): Name of the JSON file to save the response. Defaults to 'json_response_gemini'.

    Returns:
        str, str : Pickup and delivery addresses as strings
    """
    
    my_file = genai.upload_file(img_path)

    model = genai.GenerativeModel('gemini-1.5-pro')
    result = model.generate_content(
        [my_file, '\n\n', 'Extract the pickup and delivery addresses from this BOL image: return as JSON file with "pickup" and "delivery" with properties: street, city, state and zip code'],
    )
    
    result_text = result.text.strip('```').replace('json\n', '')
    json_response = json.loads(result_text)

    with open(f'{json_filename}.json', 'w') as outfile:
        json.dump(json_response, outfile, indent=2)

    pickup_addr, delivery_addr = convert_json_to_str_addr(json_addresses=json_response)
    
    return pickup_addr, delivery_addr

In [25]:
pickup, delivery = get_pickup_delivery_addr_gemini(img_path=bol_img)

print('Pickup address:', pickup)
print('Delivery address:', delivery)

Pickup address: 6115 FM 1405 BAYTOWN TX 77523
Delivery address: 10300 COURSEY BLVD BATON ROUGE LA 70816


## 4-Get Pickup and Delivery Coordinates using Google Maps's Geocoding API

Use Google Maps Platform - Geocoding API to obtain latitude and longitude values for the Pickup and Delivery addresses retrieved from the BOL.

* `get_lat_long_vals(addresses_list:list)`: Returns a list of tuples containing the latitude and longitude values for the given list of addresses.

In [26]:
def get_addresses_coords(addresses_list:list):
    """Returns a list of tuples containing the latitude and longitude values for the given list of addresses.

    Args:
        addresses_list (list): List of addresses to be converted into latitude and longitude values

    Returns:
        list: List with the latitude and longitude values for each address in the input list
    """
    coordinates_list = []
    for addr in addresses_list:
        geocode_result = gmaps.geocode(addr)
        lat_long = geocode_result[0]['geometry']['location']
        lat = lat_long['lat']
        long = lat_long['lng']
        coordinates_list.append((lat, long))

    return coordinates_list

In [27]:
pickup_coord, delivery_coord = get_addresses_coords(addresses_list=[pickup, delivery])

print('Pickup address coordinates:', pickup_coord)
print('Delivery address coordinates:', delivery_coord)

Pickup address coordinates: (29.732166, -94.917884)
Delivery address coordinates: (30.4156083, -91.06593869999999)


### Check if Trucks passed near Pickup and Delivery points

Use the methods `get_gps_location`, `get_gps_date` and `get_geo_distance_kms` to check locations, times and distances, respectively for each of the locations in the provided JSON file.

If `dist_pickup` or `dist_delivery` are less than (`<`) the parameter `dist_thresh_kms`, the Truck is considered to be at the Pickup/Delivery point and both distance and a boolean (`True`/`False`) value is recorded.

* `check_truck_passed(locations_list:list, pickup_coordinates:tuple, delivery_coordinates:tuple, dist_thresh_kms:float=0.1)`: Returns a list of dictionaries with an 'id', 'location', 'date', 'pickup' and 'delivery' keys.

In [28]:
def check_truck_passed(locations_list:list, 
                       pickup_coordinates:tuple,
                       delivery_coordinates:tuple,
                       dist_thresh_kms:float=0.5):
    """Returns a list of dictionaries with an 'id', 'location', 'date', 
    'pickup' and 'delivery' keys. The 'pickup' and 'delivery' keys are 
    dictionaries with keys 'distance_to' and 'passed_pickup'/'passed_delivery'.
    'passed_pickup' and 'passed_delivery' are True/False if the truck has(n't) 
    passed the Pickup and Delivery locations, respectively, according to the 
    dist_thresh_kms threshold parameter.
    'distance_to' is the distance between the truck and the Pickup or Delivery 
    location in kilometers.
    'id' is a unique identifier for each location.
    'location' is a tuple of (latitude, longitude) coordinates.
    'date' is the date and time of the location.

    Args:
        locations_list (list): List of locations provided in JSON file
        pickup_coordinates (tuple): Pickup location coordinates
        delivery_coordinates (tuple): Delivery location coordinates
        dist_thresh_kms (float, optional): Threshold to determine whether a truck 
        has passed or not the Pickup/Delivery points. Defaults to 0.1.

    Returns:
        list: A list of dictionaries with checks on each location provided
    """
    check_locations = []
    n_pickups = 0
    n_deliveries = 0
    for ind, loc in enumerate(locations_list):
        check = {}

        check['id'] = f'location-{ind}'
        check['pickup'] = {}
        check['delivery'] = {}

        check['location'] = get_gps_location(loc)
        check['date'] = get_gps_date(loc)
        
        dist_pickup = get_geo_distance_kms(point_a=pickup_coordinates, 
                                            point_b=check['location'])

        dist_delivery = get_geo_distance_kms(point_a=delivery_coordinates, 
                                            point_b=check['location'])
        
        check['pickup']['distance_to'] = dist_pickup
        check['delivery']['distance_to'] = dist_delivery
        
        if dist_pickup < dist_thresh_kms: 
            check['pickup']['passed_pickup'] = True
            n_pickups += 1
        else:
            check['pickup']['passed_pickup'] = False

        if dist_delivery < dist_thresh_kms:
            check['delivery']['passed_delivery'] = True
            n_deliveries += 1
        else:
            check['delivery']['passed_delivery'] = False

        check_locations.append(check)

        print('id:', ind, 'Pickups:', n_pickups, 'Deliveries:', n_deliveries)
    return check_locations

In [29]:
# Test the method to check distances (first 5 locations)
test_1 = check_truck_passed(locations_list=locations[:5], 
                             pickup_coordinates=pickup_coord, 
                             delivery_coordinates=delivery_coord,
                             dist_thresh_kms=0.1)

print(test_1)

id: 0 Pickups: 0 Deliveries: 0
id: 1 Pickups: 0 Deliveries: 0
id: 2 Pickups: 0 Deliveries: 0
id: 3 Pickups: 0 Deliveries: 0
id: 4 Pickups: 0 Deliveries: 0
[{'id': 'location-0', 'pickup': {'distance_to': 43.70951, 'passed_pickup': False}, 'delivery': {'distance_to': 409.4418, 'passed_delivery': False}, 'location': (29.96181, -95.28558), 'date': datetime.datetime(2024, 9, 7, 3, 9, 54, 153000, tzinfo=datetime.timezone.utc)}, {'id': 'location-1', 'pickup': {'distance_to': 43.83027, 'passed_pickup': False}, 'delivery': {'distance_to': 409.23432, 'passed_delivery': False}, 'location': (29.965555, -95.28402), 'date': datetime.datetime(2024, 9, 7, 3, 10, 27, 601000, tzinfo=datetime.timezone.utc)}, {'id': 'location-2', 'pickup': {'distance_to': 44.08675, 'passed_pickup': False}, 'delivery': {'distance_to': 409.45863, 'passed_delivery': False}, 'location': (29.966496, -95.28652), 'date': datetime.datetime(2024, 9, 7, 3, 11, 5, 868000, tzinfo=datetime.timezone.utc)}, {'id': 'location-3', 'pickup'

In [30]:
# Test the method to check distances (last 5 locations)
test_2 = check_truck_passed(locations_list=locations[-5:], 
                             pickup_coordinates=pickup_coord, 
                             delivery_coordinates=delivery_coord,
                             dist_thresh_kms=0.1)

print(test_2)

id: 0 Pickups: 0 Deliveries: 0
id: 1 Pickups: 0 Deliveries: 0
id: 2 Pickups: 0 Deliveries: 0
id: 3 Pickups: 0 Deliveries: 0
id: 4 Pickups: 0 Deliveries: 1
[{'id': 'location-0', 'pickup': {'distance_to': 271.97395, 'passed_pickup': False}, 'delivery': {'distance_to': 107.053, 'passed_delivery': False}, 'location': (30.247643, -92.16221), 'date': datetime.datetime(2024, 9, 10, 0, 6, 15, 986000, tzinfo=datetime.timezone.utc)}, {'id': 'location-1', 'pickup': {'distance_to': 270.48502, 'passed_pickup': False}, 'delivery': {'distance_to': 108.54847, 'passed_delivery': False}, 'location': (30.247765, -92.178024), 'date': datetime.datetime(2024, 9, 10, 0, 6, 45, 623000, tzinfo=datetime.timezone.utc)}, {'id': 'location-2', 'pickup': {'distance_to': 269.56735, 'passed_pickup': False}, 'delivery': {'distance_to': 109.4707, 'passed_delivery': False}, 'location': (30.247828, -92.18777), 'date': datetime.datetime(2024, 9, 10, 0, 7, 22, 294000, tzinfo=datetime.timezone.utc)}, {'id': 'location-3', 'pi

In [31]:
# Check for all locations
results = check_truck_passed(locations_list=locations,
                             pickup_coordinates=pickup_coord,
                             delivery_coordinates=delivery_coord,
                             dist_thresh_kms=0.5)

print(len(locations))
print(len(results))

id: 0 Pickups: 0 Deliveries: 0
id: 1 Pickups: 0 Deliveries: 0
id: 2 Pickups: 0 Deliveries: 0
id: 3 Pickups: 0 Deliveries: 0
id: 4 Pickups: 0 Deliveries: 0
id: 5 Pickups: 0 Deliveries: 0
id: 6 Pickups: 0 Deliveries: 0
id: 7 Pickups: 0 Deliveries: 0
id: 8 Pickups: 0 Deliveries: 0
id: 9 Pickups: 0 Deliveries: 0
id: 10 Pickups: 0 Deliveries: 0
id: 11 Pickups: 0 Deliveries: 0
id: 12 Pickups: 0 Deliveries: 0
id: 13 Pickups: 0 Deliveries: 0
id: 14 Pickups: 0 Deliveries: 0
id: 15 Pickups: 0 Deliveries: 0
id: 16 Pickups: 0 Deliveries: 0
id: 17 Pickups: 0 Deliveries: 0
id: 18 Pickups: 0 Deliveries: 0
id: 19 Pickups: 0 Deliveries: 0
id: 20 Pickups: 0 Deliveries: 0
id: 21 Pickups: 0 Deliveries: 0
id: 22 Pickups: 0 Deliveries: 0
id: 23 Pickups: 0 Deliveries: 0
id: 24 Pickups: 0 Deliveries: 0
id: 25 Pickups: 0 Deliveries: 0
id: 26 Pickups: 0 Deliveries: 0
id: 27 Pickups: 0 Deliveries: 0
id: 28 Pickups: 0 Deliveries: 0
id: 29 Pickups: 0 Deliveries: 0
id: 30 Pickups: 0 Deliveries: 0
id: 31 Pickups: 0 

In [32]:
one_km = (42, 328) # picks, delivs when dist_thresh_kms = 1km 
half_km = (40, 327) # picks, delivs when dist_thresh_kms = 0.5km

### Check the time Trucks stayed at Pickup and Delivery points

Sum the total elapsed time spent at Pickup and Delivery points using the key `'date'` for the `results` location checks list.

* `get_times_analysis(location_checks:list, analysis_id:float, time_threshold_sec:float=300)`: Returns total elapsed time at Pickup and Delivery locations

In [33]:
def get_times_analysis(location_checks:list,
                       analysis_id:float,
                       time_threshold_sec:float=300):
    """Returns total elapsed time at Pickup and Delivery locations. This function takes a 
    list of dictionaries containing information about the Pickup and Delivery locations, and 
    their respective times. It iterates through the list, calculating the time elapsed between 
    each pickup and delivery location. 

    Args:
        location_checks (list): A list of dictionaries containing information about the Pickup 
        and Delivery locations, and their respective times.
        analysis_id (float): A unique identifier for the analysis.
        time_threshold_sec (float, optional): The minimum time (in seconds) required for the
        total time at a location to be considered significant. Defaults to 300.

    Returns:
        dict: A dictionary containing the analysis results. The dictionary has the following
        structure:
        {
            '_id': analysis_id,
            'pickup': {
                'checked': bool,
                'total_time_sec': float
            },
            'delivery': {
                'checked': bool,
                'total_time_sec': float
            }
        }
    """
    analysis = {'_id': analysis_id,
                'pickup': {}, 
                'delivery': {}}

    prev_dt = location_checks[0]['date']

    total_pickup_time = 0
    total_delivery_time = 0
    for ind, item in enumerate(location_checks):

        if item['pickup']['passed_pickup'] == True:
            
            total_pickup_time += (item['date'] - prev_dt).total_seconds()
            print('id:', ind, 'pickup time', total_pickup_time)

        
        if item['delivery']['passed_delivery'] == True:

            total_delivery_time += (item['date'] - prev_dt).total_seconds()
            print('id:', ind, 'delivery time', total_delivery_time)

        prev_dt = item['date']

    if total_pickup_time > time_threshold_sec:
        analysis['pickup']['checked'] = True
        analysis['pickup']['total_time_sec'] = round(total_pickup_time, 3)
        
    if total_delivery_time > time_threshold_sec:
        analysis['delivery']['checked'] = True
        analysis['delivery']['total_time_sec'] = round(total_delivery_time, 3)

    return analysis        

In [34]:
analysis_results = get_times_analysis(location_checks=results,
                                      analysis_id=_id,
                                      time_threshold_sec=300)

total_pickup_time = analysis_results['pickup']['total_time_sec']
total_delivery_time = analysis_results['delivery']['total_time_sec']

id: 528 pickup time 35.009
id: 529 pickup time 114.518
id: 530 pickup time 135.93200000000002
id: 531 pickup time 159.721
id: 532 pickup time 187.791
id: 533 pickup time 224.67
id: 534 pickup time 255.91199999999998
id: 535 pickup time 286.64
id: 536 pickup time 325.62399999999997
id: 537 pickup time 381.65999999999997
id: 538 pickup time 414.40999999999997
id: 539 pickup time 481.277
id: 540 pickup time 642.71
id: 541 pickup time 655.523
id: 542 pickup time 669.7710000000001
id: 543 pickup time 705.6610000000001
id: 544 pickup time 747.176
id: 545 pickup time 822.538
id: 546 pickup time 840.267
id: 547 pickup time 872.7570000000001
id: 548 pickup time 907.652
id: 549 pickup time 933.0350000000001
id: 550 pickup time 961.5520000000001
id: 551 pickup time 996.5250000000001
id: 552 pickup time 1034.9360000000001
id: 553 pickup time 1062.93
id: 554 pickup time 1090.1370000000002
id: 555 pickup time 1118.3190000000002
id: 556 pickup time 1151.5210000000002
id: 557 pickup time 1180.62300000

In [35]:
print('Total Pickup time:', total_pickup_time)
print('Total Delivery time:', total_delivery_time)

Total Pickup time: 1585.149
Total Delivery time: 9988.553


In [36]:
# At Pickup
hours = total_pickup_time // 3600
minutes = (total_pickup_time % 3600) // 60
seconds = total_pickup_time % 60

print(f'Truck stayed at Pickup point during {hours} hrs, {minutes} min, and {seconds:.3f} s')

Truck stayed at Pickup point during 0.0 hrs, 26.0 min, and 25.149 s


In [37]:
# At Delivery
hours = total_delivery_time // 3600
minutes = (total_delivery_time % 3600) // 60
seconds = total_delivery_time % 60

print(f'Truck stayed at Pickup point during {hours} hrs, {minutes} min, and {seconds:.3f} s')

Truck stayed at Pickup point during 2.0 hrs, 46.0 min, and 28.553 s


### Analysis Results

We can conclude that, according to location traceability given in the input JSON file, **the truck stayed at least at 500m near both the Pickup and Delivery points**.

According to the analysis performed, the truck stayed:

* At Pickup location: 0.0 hrs, 26.0 min, and 25.149 s

* At Delivery location: 2.0 hrs, 46.0 min, and 28.553 s

In [38]:
with open('analysis_results.json', 'w') as file:
    json.dump(analysis_results, file, indent=2)